# TRABAJO DE PYTHON

### Este trabajo cuenta el 30% de la nota. 

### Resuelve los siguientes ejercicios en Python

#### Utiliza tantas celdas de código como necesites

#### 1.- Busca la secuencia del gen CNTNAP2 (cromosma 7) del genoma humano. Descarga la secuencia en formato FASTA e introdúcela en este Notebook . (1 pto)

In [9]:
import requests

ensembl_id = "ENSG00000174469" 
url = f"https://rest.ensembl.org/sequence/id/{ensembl_id}?type=genomic"
headers = {"Content-Type": "text/x-fasta"}  

In [10]:
r = requests.get(url, headers=headers, timeout=60)

In [12]:
if not r.ok:
    r.raise_for_status()

In [14]:
fasta_path = "CNTNAP2_ensembl_genomic.fasta"
with open(fasta_path, "wb") as f:
    f.write(r.content)

In [15]:
print(f"Guardado en: {fasta_path}\nPrimeras 10 líneas:")
with open(fasta_path) as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 9:
            break

Guardado en: CNTNAP2_ensembl_genomic.fasta
Primeras 10 líneas:
>ENSG00000174469.23 chromosome:GRCh38:7:146116002:148420998:1
GCTGATGGACCCCGGCCGGTCGGCTCAGTCTCCCTTTTGCTTTCAAACGAGGCCAGGGCA
GGGCAGAAGGGTTTTGACAGCCAGAGGGGTACGGTAAGGAGGAGGAAGAGGAGGAGGAGG
CGGAGGAGGAGGAGGAGCCGAGGAGGGAACCGAGAAGGGAAGAGAAGGGAGGGCTTCCCT
CCGCCTCGCTCGCCGCCCGGGCTCACAGGAGCTGGCGGCGGCGGCGGCGGCGGCGGCGTC
TCCTGCTCTCCGCGGCTGTTGCTGCTGCTCGCGCTCCGCCGCCCGGGAGATGCTTCCTCG
CGCGGCGCAGCGCTGAGGCCGTGCGTGCGCCCCGGCTGCGCTGCGCGCTCCCCACATACA
CAAGCTCTCCATGTGAGCTGACAGGCGAGTGGAAACCCCTCGAGTCACGCTGCCCGGCGG
CGGAGGGAGCGCTCGCCCGCAGTGGCAACAGCTGCACCACCGTCCCCGTCGCTCTGCCTT
CCTCTTCTGCAGCCTCTGCTCTTCTGATTACCTCCCTCCCCCGTCCTTTGGTGATTTTTT


#### 2.- Muestre el encabezado y la secuencia y guarde cada uno en variables separadas. Elimine el elemento "\n" del encabezado con replace() si es que lo tiene. ¿Cual es la longitud de la secuencia?. (1 pto)

In [22]:
fasta_path = "CNTNAP2_ensembl_genomic.fasta"

In [23]:
with open(fasta_path) as f:
    lines = f.readlines()

In [24]:
header = lines[0].replace("\n", "")
sequence = "".join(line.strip() for line in lines[1:])

In [25]:
print("Encabezado:", header)
print("Primeros 100 nt de la secuencia:", sequence[:100])
print("Longitud total de la secuencia:", len(sequence))

Encabezado: >ENSG00000174469.23 chromosome:GRCh38:7:146116002:148420998:1
Primeros 100 nt de la secuencia: GCTGATGGACCCCGGCCGGTCGGCTCAGTCTCCCTTTTGCTTTCAAACGAGGCCAGGGCAGGGCAGAAGGGTTTTGACAGCCAGAGGGGTACGGTAAGGA
Longitud total de la secuencia: 2304997


#### 3.- Para el encabezado, extrae empleando expresiones regulares el identificador. (1pto)

In [29]:
import re

In [30]:
match = re.match(r"^>(\S+)", header)
if match:
    identifier = match.group(1)
    print("Identificador extraído:", identifier)

Identificador extraído: ENSG00000174469.23


#### 4.- ¿Cuál es el porcentaje de GC de la secuencia? (1 pto)

In [31]:
g_count = sequence.upper().count("G")
c_count = sequence.upper().count("C")
gc_total = g_count + c_count

In [32]:
seq_length = len(sequence)

In [33]:
gc_percent = (gc_total / seq_length) * 100

In [34]:
print("Longitud de la secuencia:", seq_length)
print("Número de G:", g_count)
print("Número de C:", c_count)
print("Contenido GC (%):", round(gc_percent, 2))

Longitud de la secuencia: 2304997
Número de G: 435907
Número de C: 429028
Contenido GC (%): 37.52


#### 5.- Devuelve la secuencia de ARN que surgiría a partir de la siguiente secuencia. Recuerda que la secuencia de ARN resultante tiene que ser complementaria a la de ADN. (1 pto)

In [35]:
dna_to_rna = {
    "A": "U",
    "T": "A",
    "G": "C",
    "C": "G"  }

In [36]:
rna_seq = "".join(dna_to_rna.get(base, "") for base in sequence.upper())

In [38]:
print("Primera parte de la secuencia de ARN:", rna_seq[:100])
print("Longitud de la secuencia de ARN igual a la de ADN:", len(rna_seq))

Primera parte de la secuencia de ARN: CGACUACCUGGGGCCGGCCAGCCGAGUCAGAGGGAAAACGAAAGUUUGCUCCGGUCCCGUCCCGUCUUCCCAAAACUGUCGGUCUCCCCAUGCCAUUCCU
Longitud de la secuencia de ARN igual a la de ADN: 2304997


#### 6.- Realiza una función que realice una digestión con las siguientes enzimas de restricción en nuestra secuencia. La función tiene que devolver la siguiente información: (2 pto)
1. Enzimas que tienen sitios de corte en la secuencia y posición (o posiciones de corte)
2. Enzimas que no tienen sitios de corte (si las hay)
3. Resultado de la digestión: los fragmentos resultantes de nuestra secuencia
4. Indicar número y tamaño de cada fragmento

In [45]:
enzR = {
    'HindIII' : 'A*AGCTT',
    'BamHI' : 'G*GATCC',
    'AluI' : 'AG*CT',
    'Sau3AI' : '*GATC',
    'EcoRI' : 'G*AATTC'
}

In [47]:
def digestion_adn(secuencia, enzR):
    secuencia = secuencia.upper()
    cortes_info = {}
    sin_cortes = []
    posiciones_corte = []

    for enzima, patron in enzR.items():
        sitio = patron.replace('*', '')
        pos_corte = patron.find('*')
        posiciones = []
        inicio = 0

        while True:
            idx = secuencia.find(sitio, inicio)
            if idx == -1:
                break
            corte_real = idx + pos_corte
            posiciones.append(corte_real)
            inicio = idx + 1

        if posiciones:
            cortes_info[enzima] = posiciones
            posiciones_corte.extend(posiciones)
        else:
            sin_cortes.append(enzima)

    if not posiciones_corte:
        fragmentos = [secuencia]
    else:
        posiciones_corte = sorted(posiciones_corte)
        fragmentos = []
        inicio = 0
        for corte in posiciones_corte:
            fragmentos.append(secuencia[inicio:corte])
            inicio = corte
        fragmentos.append(secuencia[inicio:])

    print(" Enzimas que cortan y posiciones:")
    for enzima, pos in cortes_info.items():
        print(f"  {enzima}: {pos}")

    print("\n Enzimas que no tienen sitios de corte:")
    if sin_cortes:
        print("  " + ", ".join(sin_cortes))
    else:
        print("  Ninguna")

    print("\n Fragmentos resultantes:")
    for i, frag in enumerate(fragmentos, 1):
        print(f"  Fragmento {i}: {frag} (tamaño: {len(frag)})")

    print(f"\nNúmero total de fragmentos: {len(fragmentos)}")
    return fragmentos

secuencia = "AAGCTTGATCCGGAATTCCAGCTGATCGGATCC"
fragmentos = digestion_adn(secuencia, enzR)


 Enzimas que cortan y posiciones:
  HindIII: [1]
  BamHI: [28]
  AluI: [3, 21]
  Sau3AI: [6, 23, 28]
  EcoRI: [13]

 Enzimas que no tienen sitios de corte:
  Ninguna

 Fragmentos resultantes:
  Fragmento 1: A (tamaño: 1)
  Fragmento 2: AG (tamaño: 2)
  Fragmento 3: CTT (tamaño: 3)
  Fragmento 4: GATCCGG (tamaño: 7)
  Fragmento 5: AATTCCAG (tamaño: 8)
  Fragmento 6: CT (tamaño: 2)
  Fragmento 7: GATCG (tamaño: 5)
  Fragmento 8:  (tamaño: 0)
  Fragmento 9: GATCC (tamaño: 5)

Número total de fragmentos: 9


#### 7.- Realiza una función que mediante la introducción de una secuencia de ADN devuelva lo siguiente: (3pto)
1. Todos los ORFs posibles que puedan surgir de la secuencia.
2. Selecciona los 6 ORFs más largos e indica si corresponden a la strand positiva o negativa.
3. Devuelve como mejor predicción, el más largo de todos.
4. Calcula el porcentaje de aminoácidos aromáticos de esa proteína elegida y su longitud relativa a la secuencia original de ADN.

In [50]:
codon_table = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                 
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*',
    'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W',
}

In [51]:
def rev_comp(seq):
    comp = str.maketrans('ATCG', 'TAGC')
    return seq.translate(comp)[::-1]

def traducir(seq):
    prot = ""
    for i in range(0, len(seq)-2, 3):
        codon = seq[i:i+3]
        aa = codon_table.get(codon, 'X')  
        prot += aa
    return prot


In [52]:
def encontrar_orfs(secuencia):
    secuencia = secuencia.upper()
    start_codon = "ATG"
    stop_codons = ["TAA", "TAG", "TGA"]
    orfs = []

    for frame in range(3):
        seq = secuencia[frame:]
        for i in range(0, len(seq)-2, 3):
            codon = seq[i:i+3]
            if codon == start_codon:
                for j in range(i, len(seq)-2, 3):
                    codon_stop = seq[j:j+3]
                    if codon_stop in stop_codons:
                        orf_seq = seq[i:j+3]
                        orfs.append({
                            "orf_seq": orf_seq,
                            "pos": (frame + i, frame + j + 3),
                            "frame": f"+{frame+1}"
                        })
                        break

    secuencia_rev = rev_comp(secuencia)
    for frame in range(3):
        seq = secuencia_rev[frame:]
        for i in range(0, len(seq)-2, 3):
            codon = seq[i:i+3]
            if codon == start_codon:
                for j in range(i, len(seq)-2, 3):
                    codon_stop = seq[j:j+3]
                    if codon_stop in stop_codons:
                        orf_seq = seq[i:j+3]
                        orfs.append({
                            "orf_seq": orf_seq,
                            "pos": (frame + i, frame + j + 3),
                            "frame": f"-{frame+1}"
                        })
                        break

    return orfs

In [53]:
def analizar_orfs(secuencia):
    orfs = encontrar_orfs(secuencia)
    
    for o in orfs:
        o['prot_seq'] = traducir(o['orf_seq'])
        o['longitud'] = len(o['prot_seq'])

    orfs_ordenados = sorted(orfs, key=lambda x: x['longitud'], reverse=True)

    top6 = orfs_ordenados[:6]

    print(" Top 6 ORFs más largos:")
    for i, o in enumerate(top6, 1):
        print(f"{i}. Hebra {o['frame']} | Longitud: {o['longitud']} aa | Posición: {o['pos']}")

    mejor = orfs_ordenados[0]
    prot = mejor['prot_seq']
    arom = sum(1 for aa in prot if aa in "FYW")  # Aromáticos
    pct_arom = (arom / len(prot))*100 if len(prot) > 0 else 0
    longitud_rel = (len(mejor['orf_seq']) / len(secuencia))*100

    print("\n Mejor predicción:")
    print(f"Hebra: {mejor['frame']}")
    print(f"Longitud ORF: {len(mejor['orf_seq'])} nt ({mejor['longitud']} aa)")
    print(f"Secuencia proteína: {prot}")
    print(f"% aminoácidos aromáticos: {pct_arom:.2f}%")
    print(f"Longitud relativa al ADN original: {longitud_rel:.2f}%")

    return top6, mejor

In [29]:
secuencia = "ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG"
top6, mejor = analizar_orfs(secuencia)


🔹 Top 6 ORFs más largos:
1. Hebra +1 | Longitud: 8 aa | Posición: (0, 24)
2. Hebra +1 | Longitud: 4 aa | Posición: (12, 24)

🔹 Mejor predicción:
Hebra: +1
Longitud ORF: 24 nt (8 aa)
Secuencia proteína: MAIVMGR*
% aminoácidos aromáticos: 0.00%
Longitud relativa al ADN original: 61.54%
